# Imports

In [1]:
# <your imports>
import numpy as np
import torch
from torch.optim import Adam
import torch.nn as nn
from tqdm import tqdm
from omegaconf import OmegaConf
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import TrainingArguments, Trainer
from transformers import BertForSequenceClassification, XLMRobertaXLConfig
from transformers import EarlyStoppingCallback

from preprocessing import Preprocessing


# models
from models import BertClassifier

# ignore fucking warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
def compute_metrics(p):
    pred, labels = p
    # print(pred, labels)
    # print(np.unique(labels, return_counts=True), labels.shape)
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='weighted')
    precision = precision_score(y_true=labels, y_pred=pred, average='weighted')
    f1 = f1_score(y_true=labels, y_pred=pred, average='weighted')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Data loading/preprocessing

In [10]:
preprocessor = Preprocessing()

# result = preprocessor.get_dataloaders()
result = preprocessor.get_datasets()

train_set = result["train_set"]
val_set = result["val_set"]

In [11]:
np.unique(train_set.labels), np.unique(val_set.labels)

(array([0, 1, 2]), array([0, 1, 2]))

# Model initialization

In [12]:
conf = OmegaConf.load("config.yaml").general
for i in conf.keys():
    print(f'{i}: {conf[i]}')

dataset_name: ruSentNE
batch_size: 64
device: cpu:0
dataloader_shuffle: True
pretrained_model: XLMRobertaXLModel
num_classes: 3
num_workers: 1
lr: 1e-06
epochs: 5
eps: 1e-08
optimizer: AdamW
eval_steps: 10
logging_steps: 10


In [13]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "cardiffnlp/twitter-roberta-base-sentiment", 
    # 'cardiffnlp/twitter-xlm-roberta-base-sentiment',
    num_labels=3
)

In [14]:
# # model = BertForSequenceClassification.from_pretrained(conf.pretrained_model,
# #                                                       num_labels=conf.num_classes,
# #                                                       output_attentions=False,
# #                                                       output_hidden_states=False)

# model = XLMRobertaXLModel.from_pretrained(conf.pretrained_model,
#                                            num_labels=conf.num_classes,
#                                            output_attentions=False,
#                                            output_hidden_states=False)

In [15]:
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=conf.eval_steps,
    per_device_train_batch_size=conf.batch_size,
    per_device_eval_batch_size=conf.batch_size,
    num_train_epochs=conf.epochs,
    seed=42,
    logging_steps=conf.logging_steps,
    # learning_rate=conf.lr,
    learning_rate=5e-7,
    load_best_model_at_end=True,
)

In [16]:
args.learning_rate

5e-07

In [17]:
# help(Trainer)

In [18]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_set,
    eval_dataset=val_set,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

## Train loop

In [19]:
# API key for wandb: 1fef7fd74ee7f9dfeb33b86d13b60f8ffe9c968a

In [ ]:
trainer.train()

# Custom Trainer

In [6]:
from trainer import CustomTrainer

In [7]:
preprocessor = Preprocessing()

result = preprocessor.get_dataloaders()

train_dl = result["train_dl"]
val_dl = result["val_dl"]

In [8]:
trainer = CustomTrainer(model.cuda())

Initialized training config with params: {'dataset_name': 'ruSentNE', 'batch_size': 64, 'device': 'cpu:0', 'dataloader_shuffle': True, 'pretrained_model': 'XLMRobertaXLModel', 'num_classes': 3, 'num_workers': 1, 'lr': 1e-06, 'epochs': 5, 'eps': 1e-08, 'optimizer': 'AdamW', 'eval_steps': 10, 'logging_steps': 10}


In [9]:
trainer.train(train_dl, val_dl)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/119 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.869925977803078
Validation loss: 0.7924036224683125
F1 Score (Weighted): 0.6015068903481781


Epoch 2:   0%|          | 0/119 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.7996472861586499
Validation loss: 0.7991720914840699
F1 Score (Weighted): 0.6015068903481781


...


Epoch 3
Training loss: 0.7904887595096556
Validation loss: 0.7876230895519256
F1 Score (Weighted): 0.6015068903481781


Epoch 4:   0%|          | 0/119 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.7902478320258004
Validation loss: 0.783882341782252
F1 Score (Weighted): 0.6015068903481781


Epoch 5:   0%|          | 0/119 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.787754876273019
Validation loss: 0.7829690237840017
F1 Score (Weighted): 0.6015068903481781


# Experiments